## London Underground network connections map

#### Function definitions for network map (to be run once, but not to be changed)

Define function to specify colour for each group.

In [ ]:
def group_colour(group, colourscheme):
    # if member belongs to multiple groups return a neutral colour
    if group == 'Multiple Groups':
        return 'white'
    else:
        # find group in colourscheme dataset
        for i in range(0, len(colourscheme.iloc[:,0])):
            if group == colourscheme.iloc[i,0]:
                return colourscheme.iloc[i,1]
        
        # return value in case there is no match
        return 'white'

Define functions to find 'graph' (mathematical name for a nextwork map) nodes and edges.

In [ ]:
def graph_nodes(dataset, colourscheme):
    # create empty lists to store node names and membership 
    Nodes = list()
    Colours = list()
    
    # add information about each node (member of groups)
    for i in range(0, len(dataset.iloc[:,0])):
        group = 'None'
        # check if node belongs to multiple groups
        for j in range(1, len(dataset.columns)):
            # check if member is in group
            if dataset.iloc[i,j] > 0:
                # check if this is the first group the member is in
                if group == 'None':
                    group = dataset.columns[j]
                else:
                    group = 'Multiple Groups'
        # add node name and membership to list
        Nodes.append(dataset.iloc[i,0])
        Colours.append(group_colour(group, colourscheme))
    
    # return list of members in all groups
    return Nodes, Colours

In [ ]:
def graph_edges(dataset):
    # create empty list to store edges
    Edges = list()
    
    # add edges between members in each group
    for i in range(1, len(dataset.columns)):
        # cycle through each member in the group
        for j in range(0, len(dataset.iloc[:,i])):
            # if member is in the group, add edges to other members in the group
            if dataset.iloc[j,i] > 0:
                for k in range(j + 1, len(dataset.iloc[:,i])):
                    # if second member is also in the group add edge between them
                    if dataset.iloc[k,i] > 0:
                        Edges.append((j, k))
    
    # return filled list of edges between members in each of the groups
    return Edges

Define function to create a network plot given the 'graph' nodes and edges derived for the dataset.

In [ ]:
import igraph as ig
import plotly.offline as py
import plotly.graph_objs as go

def network_plot(dataset, colourscheme, filename):
    # find node names and colours from dataset
    Nodes, Colours = graph_nodes(dataset, colourscheme)
    N = len(Nodes)

    # calculate 'graph' edges from dataset
    Edges = graph_edges(dataset)
    G = ig.Graph(Edges, directed=False)

    # define the graph as a 3 dimensional object
    layt = G.layout('kk', dim=3)

    # define coordinates of nodes
    Xn = [layt[k][0] for k in range(N)] # x-coordinates of nodes
    Yn = [layt[k][1] for k in range(N)] # y-coordinates
    Zn = [layt[k][2] for k in range(N)] # z-coordinates

    # define coordinates of each end of edges
    Xe = []
    Ye = []
    Ze = []
    for e in Edges:
        Xe += [layt[e[0]][0],layt[e[1]][0], None] # x-coordinates of edge ends
        Ye += [layt[e[0]][1],layt[e[1]][1], None]
        Ze += [layt[e[0]][2],layt[e[1]][2], None]

    # plot the edges of the 'graph'
    trace1 = go.Scatter3d(x=Xe, y=Ye, z=Ze, mode='lines', line=dict(color='rgb(125,125,125)', width=0.5), \
                    hoverinfo='none')

    # plot the nodes of the 'graph'
    trace2 = go.Scatter3d(x=Xn, y=Yn, z=Zn, mode='markers+text', name=dataset.columns[0], \
                    marker=dict(symbol='circle', size=6, color=Colours, line=dict(color='rgb(50,50,50)', width=1)), \
                    text=Nodes, textposition='middle right', hoverinfo='text')

    # set properties of all 3 axes
    axis = dict(showbackground=False, showline=False, zeroline=False, showgrid=False, showticklabels=False, \
                    showspikes=False, title='')

    # define plot layout
    layout = go.Layout(width=1000, height=1000, showlegend=False, \
                    scene=dict(xaxis=dict(axis), yaxis=dict(axis), zaxis=dict(axis)), margin=dict(t=0), \
                    hovermode='closest')

    # create network plot
    data = [trace1, trace2]
    fig = go.Figure(data=data, layout=layout)
    py.plot(fig, config={'scrollzoom': False}, filename=filename, auto_open=False)

#### Import dataset of nodes and group membership, and colourscheme to apply to each group (change file location as desired)

Import the London Underground connections dataset from `github` repository into a `pandas` dataframe. Note this reads in the `.csv` version of the dataset directly from `github`.

This dataset is formatted so that each column (after the first) indicates membership of a particular group, in this case an underground line. All members of the group (in this case stations, listed in the first column) are assumed to be directly linked to each other.

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# read-in the dataset from github into a pandas dataframe
url = "https://raw.githubusercontent.com/xbr209/tutorials/master/London_underground.csv"
dataset = pd.read_csv(url)

# print out the pandas dataframe to screen
dataset

Import the London Underground colourscheme dataset from `github` repository into a `pandas` dataframe.

This dataset is formatted so that the first column is the group name (used in the main dataset) and the second column is the colour name to be used for that group. The colour name must be in a format recognised by `plotly`; the full range of colours can be found at https://matplotlib.org/3.1.0/gallery/color/named_colors.html.

In [ ]:
# read-in colour scheme for each group from github into a pandas dataframe
url = "https://raw.githubusercontent.com/xbr209/tutorials/master/London_underground_colourscheme.csv"
colourscheme = pd.read_csv(url)

colourscheme

#### Produce network plot from input dataset and colour scheme

Function that creates a network plot; takes the dataset, colourscheme and output filename as inputs.

In [ ]:
network_plot(dataset, colourscheme, 'london_underground')